In [1]:
# import modules
import numpy as np
import pandas as pd
import ipywidgets as widgets
from matplotlib import pyplot as plt
import matplotlib.image as img
from IPython.display import display, Markdown, clear_output
from rich.console import Console
import glob
import os
import shutil
from rich.text import Text
console = Console()


In [6]:
# open the image csv (qc_views_df)
qc_views_df = pd.read_csv('./output/cats/catalogue_20241016.csv')
qc_views_df = qc_views_df#[:6]
# image lsit
id_list = qc_views_df['grid_id'].to_list()
im_list = ['./output/images/'+item+'.png' for item in id_list]#[:10]
qc_views_df.head()

,Unnamed: 0,grid_id,Editor,day
0,0,OR_h3v3_ITC_23393_2007,Tylor,20241015
1,1,CA_h3v10_ITC_18505_2018,Tylor,20241015
2,3,CA_h3v10_ITC_18502_2018,Tylor,20241015
3,5,CA_h3v10_ITC_18649_2018,Tylor,20241015
4,6,CA_h2v6_ITC_8351_2015,Tylor,20241015


In [10]:
# empty list for QA
qa_list = []
# back list to add images use in back button
back_ls = []
# list for comments
comment_list = []
# confidance list
conf_list = []

print(f'There {len(im_list)} images to display')
# store the intial number of grids/images for checkings
original_length  = len(im_list)

# next button wideget
button = widgets.Button(description="Next view", icon='telegram')
button.style.button_color = '#EEF1F4'
button.style.text_color = '#338EF4'
output = widgets.Output()

# option button /toggle button
toggs = widgets.ToggleButtons(
    options=['Approve', 'Drop', 'Re-do'],
    description='Quality Flag',
    #disabled=False,
    #button_style='success'
    )
toggs.style.button_color = '#8E44AD'
toggs.style.text_color = '#8E44AD'

# back button; may be add onclick  fucnction to reverse the display image list
button2 = widgets.Button(description="Back", icon='step-backward')
button2.style.button_color = '#EEF1F4'
button2.style.text_color = '#338EF4'

# reset button; Execute only need  
button3 = widgets.Button(description= "Reset", icon='refresh')
button3.style.button_color = '#EEF1F4'
button3.style.text_color = '#338EF4'

# confidance level selector
conf = widgets.Select(
    options=['High_Confidence', 'Low_Confidence'],
    value='High_Confidence',
    # rows=10,
    description='Label is:',
    disabled=False
)

forward_count = 0
# progress bar
prog_bar = widgets.IntProgress(
    value=forward_count,
    min=0,
    max=original_length,
    description='Progress',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': '#46516b'}, ##3f2759, #46516b
    orientation='vertical'
)

# comment box
com_text = widgets.Textarea(
    value='no special comments',
    placeholder='No comments',
    description='Comment',
    disabled=False
)

# adding a legend 
legend_image = open('./output/legend_2.png', "rb")
image = legend_image.read()

lege_image = widgets.Image(
    value=image,
    format = 'png',
    width= 400,
    height= 300)

# track the last button clicked
last_clicked_button = None
# start with zero forward count
forward_count = 0
# for the next button click
def on_button_clicked(b):
    global forward_count, last_clicked_button
    # after click back button add one more to go from one image
    # this is basically for avoiding showing image from back and forward
    if last_clicked_button == button2:
        forward_count += 1
    # switch back to this button not to add more from back button
    last_clicked_button = button
    # show greeting images
    if forward_count >= original_length:
        plot_path = './output/logo.png'
        #print('Warning - Stop the QC process and proceed to the next step !!!')
    # forward counting and indexing
    else:
        # grab the relavant image with forward count indexing
        plot_path = im_list[forward_count]
        forward_count = forward_count + 1
    # progress bar: run with the next button to update value
    prog_bar.value = forward_count
    # outputing 
    with output:
        output.clear_output()
        # plot figure
        plt.figure(figsize=(12,12))
        im = img.imread(plot_path)
        plt.imshow(im)
        plt.axis('off')
        plt.show()
        plt.close()
        # This allows to see the first image and decide the flag after that
        if forward_count > 1 and forward_count <= original_length+1: 
            qa_list.append(toggs.value)
            conf_list.append(conf.value)
            comment_list.append(com_text.value)
        if len(qa_list) > original_length:
            qa_list.pop(-1)
            conf_list.pop(-1)
            comment_list.pop(-1)
        # show qa_list
        text = Text()
        if len(qa_list) > 0:
            #console.print(f'   previoues image flagged as - {qa_list[-1]}', style='bold')
            text.append('   previoues image flagged as - ', style='bold')
            text.append(qa_list[-1], style='bold #a33cb5')
            console.print(text)
        if len(qa_list) < original_length:
            console.print(f'   {(original_length - forward_count) } More views to go', style='bold')
        else:
            console.print('   No images to display - Proceed to the next step', style='bold')
        # defaulting the comment box
        com_text.value = 'no special comments'

# start back list        
back_count = 0
def on_button_clicked_2(c):
    global forward_count, last_clicked_button 
    # reducing forward count on back button
    forward_count = forward_count - 1
    # assigned last click button
    last_clicked_button = button2
    # default forward count to the zero
    #if len(qa_list) > 0:########################
    #    qa_list.pop(-1)
    if forward_count < 0:
        forward_count = 0
    #print(f'Back count is {forward_count}')#####
    draw_path = im_list[forward_count]
    # progree bar value
    prog_bar.value = forward_count
    # default the forward count to zero
    with output:
        output.clear_output()
        plt.figure(figsize=(12,12))
        im = img.imread(draw_path)
        plt.imshow(im)
        plt.axis('off')
        #if len(im_list) > 0:im_list.pop(0)
        plt.show()
        plt.close()
        # qa flag display and
        # previoues flag removal
        if len(qa_list) > 0:
            #console.print(f'  This view was previously QC flagged as - {qa_list[-1]}', style='bold')
            text = Text() 
            text.append('This view was previously QC flagged as - ', style='bold')
            text.append(qa_list[-1], style="bold #3cb5a5")
            console.print(text)
            
            qa_list.pop(-1)
            comment_list.pop(-1)
            conf_list.pop(-1)
        # defaulting the comment box
        com_text.value = 'no special comments'
        
# Reset button
def on_button_clicked_3(d):
    global forward_count, qa_list, comment_list
    forward_count = 0
    qa_list = []
    comment_list=[]
    conf_list = []
    print('system reset')

# assign onclick operation
button.on_click(on_button_clicked)
button2.on_click(on_button_clicked_2)
button3.on_click(on_button_clicked_3)

# too keep a space between next and legend
spacer = widgets.Label(value="", layout=widgets.Layout(width="50px"))

# display arrangements
output = widgets.Output()
button_box = widgets.HBox([button, spacer, lege_image])
togg_box = widgets.HBox([toggs])
image_box = widgets.HBox([output, prog_bar])
button_box2 = widgets.HBox([button2, button3, com_text, conf])



# Combine them into a single VBox or HBox
combined = widgets.VBox([button_box, togg_box, image_box, button_box2])

# Apply a layout with a border to the combined VBox
combined.layout = widgets.Layout(
    border='solid 3px black',  
    padding='10px',            
    margin='10px',
    width='1000px',
    height='1100px'     
)

# Display the combined widgets with the border
display(combined)
# display
#display(button_box)
#display(toggs)
#display(image_box)
#display(button_box2)

qc_list = qa_list[:original_length]

There 50 images to display


In [11]:
qa_list = qa_list                  #[:24]#[:original_length]
comment_list = comment_list        #[:24]#[:original_length]
print(qa_list)
print(comment_list)
print(conf_list)

['Drop', 'Approve', 'Drop', 'Approve', 'Drop', 'Approve', 'Drop', 'Drop', 'Drop', 'Approve', 'Drop', 'Approve', 'Approve', 'Drop', 'Approve', 'Approve', 'Approve', 'Approve', 'Approve', 'Approve', 'Approve', 'Approve', 'Drop', 'Drop', 'Approve', 'Approve', 'Approve', 'Approve', 'Approve', 'Drop', 'Approve', 'Approve', 'Drop', 'Drop', 'Approve', 'Drop', 'Approve', 'Drop', 'Approve', 'Approve', 'Approve', 'Drop', 'Approve', 'Approve', 'Approve', 'Approve', 'Drop', 'Approve', 'Approve', 'Drop']
['redo', 'no special comments', 'redo', 'no special comments', 'no special comments', 'no special comments', 'as images are not good', 'redo', 'as fire', 'at least 50% dead at the end', 'other disturbance happened', 'yes. not much dead', 'sparse vegetation', 'there are more impacted', 'no special comments', 'no special comments', 'no special comments', 'no special comments', 'no special comments', 'harvest-2022', 'no special comments', 'no special comments', 'miss classifcation', 'no special commen

In [12]:
qc_views_df = qc_views_df         #[:24]

In [13]:
len(qa_list)

50

In [14]:
qc_views_df['QC'] = qa_list
qc_views_df['comment'] = comment_list
qc_views_df

,Unnamed: 0,grid_id,Editor,day,QC,comment
0,0,OR_h3v3_ITC_23393_2007,Tylor,20241015,Drop,redo
1,1,CA_h3v10_ITC_18505_2018,Tylor,20241015,Approve,no special comments
2,3,CA_h3v10_ITC_18502_2018,Tylor,20241015,Drop,redo
3,5,CA_h3v10_ITC_18649_2018,Tylor,20241015,Approve,no special comments
4,6,CA_h2v6_ITC_8351_2015,Tylor,20241015,Drop,no special comments
5,7,CA_h2v6_ITC_8832_2015,Tylor,20241015,Approve,no special comments
6,8,OR_h3v3_ITC_23395_2007,Tylor,20241015,Drop,as images are not good
7,10,CA_h2v6_ITC_8511_2015,Tylor,20241015,Drop,redo
8,11,OR_h3v4_ITC_16490_2013,Tylor,20241015,Drop,as fire
9,12,OR_h3v4_ITC_17461_2013,Tylor,20241015,Approve,at least 50% dead at the end


In [15]:
# updated cat
qc_views_df.to_csv('./output/cats/qc_cat_2_20241016.csv')

In [15]:
# filtering pooint shape files based on that
# basically take the approved list and copy those folders to a different destination

# approved df
approved_df = qc_views_df[qc_views_df.QC == 'Approve']
# redo df
redo_df = qc_views_df[qc_views_df.QC == 'Re-do']
# usable grid list
good_grids = approved_df.grid_id.to_list()
# redo grids
redo_grids = redo_df.grid_id.to_list()


In [16]:
# simply list the folder has shape files
all  = os.listdir('./Input/points_EP_20230513')

In [17]:
# coopy files approved
# source path and target path for file copying
sp = './Input/points_EP_20230513'
tp = './Input/pass'
for file_name in all:
    for name_part in good_grids:
        if file_name.startswith(name_part):
            source_path = os.path.join(sp, file_name)
            target_path = os.path.join(tp, file_name)
            try:
                shutil.copy2(source_path, target_path)
                print('Approved files copied succefully')
            except:
                print('approved data transfered already')

# copy files redo
sp = './Input/points_EP_20230513'
tp = './Input/redo'
for file_name in all:
    for name_part in redo_grids:
        if file_name.startswith(name_part):
            source_path = os.path.join(sp, file_name)
            target_path = os.path.join(tp, file_name)
            try:
                shutil.copy2(source_path, target_path)
                print('redo files copied succefully')
            except:
                print('redo data transfered already')

Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved files copied succefully
Approved f

In [18]:
# copy relavent images to the 
all_images = os.listdir('./output/images')

# copy files redo
sp = './output/images'
tp = './Input/redo'
for file_name in all_images:
    for name_part in redo_grids:
        if file_name.startswith(name_part):
            source_path = os.path.join(sp, file_name)
            target_path = os.path.join(tp, file_name)
            try:
                shutil.copy2(source_path, target_path)
                print('redo files copied succefully')
            except:
                print('redo data transfered already')

redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
redo files copied succefully
